## 1. Source Account Stream

In [ ]:
import boto3

source_session = boto3.Session(profile_name='pa')

source_kda_client = source_session.client('kinesisanalyticsv2')
source_kinesis = source_session.client('kinesis')
source_iam = source_session.client('iam')

In [ ]:
cis = source_kinesis.create_stream(
    StreamName='ExampleInputStream',
    ShardCount=1
)

print(cis)

In [ ]:
import os
source_account = os.environ['PRODUCER_ACCOUNT_NO']

In [ ]:
rsc = source_kinesis.register_stream_consumer(
    StreamARN="arn:aws:kinesis:us-east-1:" + source_account + ":stream/ExampleInputStream",
    ConsumerName='samplecon'
)
print(rsc)

In [ ]:
fanout_stream_arn = rsc['Consumer']['ConsumerARN']
print(fanout_stream_arn)

## Source Account Permissions

In [ ]:
import os
source_account = os.environ['PRODUCER_ACCOUNT_NO']
sink_account = os.environ['CONSUMER_ACCOUNT_NO']

In [ ]:
input_stream_reader_trust_relationship="""{
    "Version": "2012-10-17",
    "Statement": [
      {
        "Effect": "Allow",
        "Principal": {
          "AWS":  "arn:aws:iam::""" + sink_account + """:user/ca"
        },
        "Action": "sts:AssumeRole"
      }
    ]
}"""

In [ ]:
stream_arn = "arn:aws:kinesis:us-east-1:" + source_account + ":stream/ExampleInputStream"
print(stream_arn)

In [ ]:
input_stream_reader_policy="""{
    "Version": "2012-10-17",
    "Statement": [
        {
            "Sid": "ReadInputStream",
            "Effect": "Allow",
            "Action": [
                "kinesis:DescribeStream",
                "kinesis:DescribeStreamSummary",
                "kinesis:GetRecords",
                "kinesis:GetShardIterator",
                "kinesis:ListShards"
            ],
            "Resource": """ + '"' + stream_arn + '"'+ """
        },
        {
            "Effect": "Allow",
            "Action": [
                "kinesis:DescribeStreamConsumer",
                "kinesis:SubscribeToShard"
            ],
            "Resource": [
                """ + '"' + stream_arn + '/consumer/samplecon",'+ """
                """ + '"' + stream_arn + '/consumer/samplecon:*"'+ """
            ]   
        }
    ]
}"""

print(input_stream_reader_policy)

In [ ]:
cr = source_iam.create_role(
    RoleName='KA-Source-Stream-Role',
    Path='/service-role/',
    AssumeRolePolicyDocument=input_stream_reader_trust_relationship
)

print(cr)

In [ ]:
cp = source_iam.create_policy(
    PolicyName='kda-stream-reader',
    Path='/service-role/',
    PolicyDocument=input_stream_reader_policy
)

print(cp)

In [ ]:
ap = source_iam.attach_role_policy(
    RoleName='KA-Source-Stream-Role',
    PolicyArn="arn:aws:iam::{}:policy/service-role/kda-stream-reader".format(source_account)
)

## Write to Stream

In [ ]:
import datetime
import json
import random
import boto3
import uuid
import time

def get_data():
    return {
        'things': [{'a':1},{'b':2}],
        'stuff':{'key':{'foo': True, 'bar': False}},
        'EVENT_TIME': datetime.datetime.now().isoformat(),
        'TICKER': random.choice(['AAPL', 'AMZN', 'MSFT', 'INTC', 'TBV']),
        'PRICE': round(random.random() * 100, 2)}

def get_cloud_event():
    return {
        "specversion" : "1.0",
        "type" : "tick",
        "source" : "sample-stream",
        "subject" : "delayed-data",
        "id" : str(uuid.uuid4()),
        "time" : datetime.datetime.now().isoformat(),
        "datacontenttype" : "application/json",
        "data" : get_data()
    }


def generate(stream_name, kinesis_client):
    while True:
        data = get_cloud_event()
        print(data)
        kinesis_client.put_record(
            StreamName=stream_name,
            Data=json.dumps(data),
            PartitionKey="partitionkey")
        time.sleep(1)

In [ ]:
# Run it
generate("ExampleInputStream", source_kinesis)

## Clean Up

### IAM

In [ ]:
source_iam.detach_role_policy(
    RoleName='KA-Source-Stream-Role',
    PolicyArn="arn:aws:iam::{}:policy/service-role/kda-stream-reader".format(source_account)
)

In [ ]:
source_iam.delete_policy(
    PolicyArn="arn:aws:iam::{}:policy/service-role/kda-stream-reader".format(source_account)
)

In [ ]:
source_iam.delete_role(
    RoleName='KA-Source-Stream-Role'
)

### Consumer App

In [ ]:
lsc = source_kinesis.list_stream_consumers(
    StreamARN=stream_arn
)
print(lsc)

consumers = lsc['Consumers']
print(consumers)

for c in consumers:
    print(c)
    dsc = source_kinesis.deregister_stream_consumer(
        StreamARN=stream_arn,
        ConsumerARN=c['ConsumerARN']
    )
    print(dsc)

### DDB Shard Lease Tables

In [ ]:
import boto3

sink_session = boto3.Session(profile_name='ca')
sink_ddb_client = sink_session.client('dynamodb')

In [ ]:
sink_ddb_client.list_tables()

In [ ]:
sink_ddb_client.delete_table(
    TableName='samplecon'
)

In [ ]:
# Check the producer account in case you were messing around in there...
source_ddb_client = source_session.client('dynamodb')

In [ ]:
sink_ddb_client.list_tables()

### Kinesis

In [ ]:
source_kinesis.delete_stream(
    StreamName='ExampleInputStream'
)